In [6]:
import numpy as np
from collections import Counter
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,SnowballStemmer,WordNetLemmatizer
import json
import os
from tqdm.notebook import tqdm,tnrange
import string
import numpy as np
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import defaultdict

nltk.download("punkt")
nltk.download("stopwords")
class InvertedIndex():
    def __init__(self):
        self.DFpostings={}
        self.tokens_files=[]

    def __str__(self):
        return "Keys:"+str(len(list(self.db.keys())))

    def stripSpecialChar(self,text):
        return ''.join(ch for ch in text if ch.isalnum() and not ch.isdigit() and ch not in string.punctuation)

    def preProcess(self,text):
        stemmer = SnowballStemmer("english")
        stopWords = set(stopwords.words('english'))

        text = text.lower()                                     # convert all text to lower case
        text_tokens = word_tokenize(text)                       # tokenizing the text

        # stemmedWords = list([stemmer.stem(word) for word in text_tokens])
        # validTokens = [i for i in stemmedWords if i not in stopWords]

        validTokens = [i for i in text_tokens if i not in stopWords]    # removing stop words

        validTokens = [self.stripSpecialChar(x) for x in validTokens]   # stripping special characters
        validTokens = [x for x in validTokens if len(x) > 1]    # Choosing only words which has length > 1
        return validTokens, set(validTokens)

    def indexFile(self,file,fileId):
        '''
        Indexes the file pass as argument to the associated fileID
        '''
        tokens, setTokens = self.preProcess(file)
        self.tokens_files.append(tokens)
        for i in setTokens:
            if i in self.DFpostings:
                self.DFpostings[i].append(fileId)
            else:
                self.DFpostings[i] = [fileId]

    def generateWordcloud(self):
        '''
        Creates a wordclound to visualize the frequence of words in the index
        '''
        frequencyDict = {}
        for key in self.db:
            frequencyDict[key] = len(self.db[key])
        wordcloud = WordCloud().generate_from_frequencies(frequencyDict)
        plt.imshow(wordcloud, interpolation="bilinear")
        plt.axis("off")
    
    def get_fileTokens(i):
        return self.tokens_files(i)

    def save(self,file):
        '''
        Save the index to a file locally
        '''
        json.dump(self.DFpostings, open(file, "w"))


[nltk_data] Downloading package punkt to C:\Users\Sajeel
[nltk_data]     Khan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Sajeel
[nltk_data]     Khan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
allFiles = os.walk("stories")
filePaths = []
for i in allFiles:
    for j in i[2]:
        filePath = i[0] + "/" + j
        filePaths.append(filePath)

json.dump(filePaths, open("mapping.json", "w"))

index = InvertedIndex()

for i,filePath in enumerate(tqdm(filePaths)):
    try:
        file = open(filePath, encoding="utf8")
        read = file.read().replace('\n', ' ')    
    except Exception as e:
        file = open(filePath, encoding="unicode_escape")
        read = file.read().replace('\n', ' ')
    file.close()
    index.indexFile(read, i)
    
import pickle 
serialized = pickle.dumps(index)
filename = 'PostingsAndTokens2.obj'
with open(filename,'wb') as file_object:
    file_object.write(serialized)

  0%|          | 0/467 [00:00<?, ?it/s]